In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
import feature_manager as fma
import rl.env_simple_crypto_trade as env
import importlib
import rl.models as rla
import config as cf
import tr_utils
import torch as th

In [2]:
importlib.reload(fma)

fm = fma.FeatureManager()
fm.import_data(symbol="BTCUSDT",timeframes=["4h","1d","1w","1mo"])

fm.build_features(
    lags = [1,1,1,1],
    features=cf.DEFAULT_INDICATORS,
)

Imported observe data 4h from ../data/BTCUSDT-4h.csv with 11760 rows
Imported observe data 1d from ../data/BTCUSDT-1d.csv with 1963 rows
Imported observe data 1w from ../data/BTCUSDT-1w.csv with 247 rows
Imported observe data 1mo from ../data/BTCUSDT-1mo.csv with 65 rows
Calculating external features ...
sma_3_10, sma_7_30, rsi7, rsi14, rsi30, cci7, cci14, cci30, dx7, dx14, dx30, hashrate, fed_rate, gold, nasdaq, sp500, google_trend, 
sma_3_10, sma_7_30, rsi7, rsi14, rsi30, cci7, cci14, cci30, dx7, dx14, dx30, 
sma_3_10, sma_7_30, rsi7, rsi14, rsi30, cci7, cci14, cci30, dx7, dx14, dx30, 
sma_3_10, rsi7, rsi14, cci7, cci14, dx7, dx14, 

Normalizing features with MaxAbs: sma_3_10_level0_lag_1, sma_7_30_level0_lag_1, rsi7_level0_lag_1, rsi14_level0_lag_1, rsi30_level0_lag_1, cci7_level0_lag_1, cci14_level0_lag_1, cci30_level0_lag_1, dx7_level0_lag_1, dx14_level0_lag_1, dx30_level0_lag_1, hashrate_level0_lag_1, fed_rate_level0_lag_1, gold_level0_lag_1, nasdaq_level0_lag_1, sp500_level0_lag

In [3]:
train = fm.df.iloc[-4000:-2000]
trade = fm.df.iloc[-2000:]

In [4]:
importlib.reload(env)
importlib.reload(cf)

env_kwargs = cf.TRADE_ENV_PARAMETER
state_space = 10 + len(fm.cols)

train_env = env.CryptoTradingEnv(
    trade_timeframe="4h",
    df = train, 
    state_space=state_space,
    indicators=fm.cols,
    **env_kwargs)

trade_env = env.CryptoTradingEnv(
    trade_timeframe="4h",
    df = trade, 
    state_space=state_space,
    indicators=fm.cols,
    **env_kwargs)

full_env = env.CryptoTradingEnv(
    trade_timeframe="4h",
    df = fm.df, 
    state_space=state_space,
    indicators=fm.cols,
    **env_kwargs)

In [5]:
len(full_env.df)

9126

In [6]:
importlib.reload(rla)
agent = rla.DRLTradeAgent(env=full_env)

In [7]:
importlib.reload(rla)

# DQN_PARAMS = {
#     "learning_rate": 1e-4,
#     "buffer_size": 100_000,  
#     "learning_starts": 100_000,
#     "batch_size": 64,
#     "tau": 1.0,
#     "gamma": 0.9999,
#     "train_freq": 4,
#     "target_update_interval": 10000,
#     "exploration_fraction": 0.3,
#     "exploration_initial_eps": 1.0,
#     "exploration_final_eps": 0.05,
# }

# catalog_name = tr_utils.get_name_with_kwargs("dqn",DQN_PARAMS)

# dqn_model = agent.get_model(
#     model_name="dqn",
#     model_kwargs=DQN_PARAMS,
#     seed=100,
#     tensorboard_log=catalog_name
# )

PPO_MODEL_PARAMS = {
    "n_steps": 9200,
    "ent_coef": 0.02,
    "learning_rate": 0.00025,
    "batch_size": 230
}

PPO_POLICY_PARAMS = {
    "activation_fn":th.nn.Tanh,
    "net_arch": dict(pi=[128], vf=[128])
}

catalog_name = tr_utils.get_name_with_kwargs("add_1layer128_tanh",PPO_MODEL_PARAMS)

ppo_model = agent.get_model(
    model_name="ppo",
    model_kwargs = PPO_MODEL_PARAMS,
    policy_kwargs= PPO_POLICY_PARAMS,
    tensorboard_log=catalog_name,
    seed = 100
)


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [8]:
ppo_model.policy_kwargs

{'activation_fn': torch.nn.modules.activation.Tanh,
 'net_arch': {'pi': [128], 'vf': [128]}}

In [9]:
selected_model = ppo_model

selected_model = agent.train_model(
    model = selected_model,
    total_timesteps = 2_000_000,
    checkpoint = True,
    catalog_name = catalog_name,
    save_frequency = 20_000,
    progress_bar = True
)

Logging to ../logs/tensorboard_log/add_1layer128_tanh_n_steps_9200_ent_coef_0.02_learning_rate_0.00025_batch_size_230/PPO_0


Output()

-------------------------------------
| rollout/           |              |
|    ep_len_mean     | 9.13e+03     |
|    ep_rew_mean     | -3.7         |
| time/              |              |
|    fps             | 223          |
|    iterations      | 1            |
|    time_elapsed    | 41           |
|    total_timesteps | 9200         |
| train/             |              |
|    reward          | 0.0023984804 |
-------------------------------------


--------------------------------------------
| rollout/                |                |
|    ep_len_mean          | 9.13e+03       |
|    ep_rew_mean          | -2.13          |
| time/                   |                |
|    fps                  | 228            |
|    iterations           | 2              |
|    time_elapsed         | 80             |
|    total_timesteps      | 18400          |
| train/                  |                |
|    approx_kl            | 0.0070958612   |
|    clip_fraction        | 0.0371         |
|    clip_range           | 0.2            |
|    entropy_loss         | -1.09          |
|    explained_variance   | 0.0147         |
|    learning_rate        | 0.00025        |
|    loss                 | -0.0254        |
|    n_updates            | 10             |
|    policy_gradient_loss | -0.00111       |
|    reward               | -0.00050012505 |
|    value_loss           | 0.00146        |
--------------------------------------------


------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 9.13e+03     |
|    ep_rew_mean          | -2.88        |
| time/                   |              |
|    fps                  | 227          |
|    iterations           | 3            |
|    time_elapsed         | 121          |
|    total_timesteps      | 27600        |
| train/                  |              |
|    approx_kl            | 0.007623489  |
|    clip_fraction        | 0.0232       |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.09        |
|    explained_variance   | -5.41e-05    |
|    learning_rate        | 0.00025      |
|    loss                 | -0.0195      |
|    n_updates            | 20           |
|    policy_gradient_loss | 8.85e-05     |
|    reward               | 0.0008002301 |
|    value_loss           | 0.00148      |
------------------------------------------


...

-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 9.13e+03      |
|    ep_rew_mean          | 0.85          |
| time/                   |               |
|    fps                  | 227           |
|    iterations           | 214           |
|    time_elapsed         | 8672          |
|    total_timesteps      | 1968800       |
| train/                  |               |
|    approx_kl            | 0.00012647893 |
|    clip_fraction        | 0.00105       |
|    clip_range           | 0.2           |
|    entropy_loss         | -0.107        |
|    explained_variance   | 6.56e-06      |
|    learning_rate        | 0.00025       |
|    loss                 | 0.00284       |
|    n_updates            | 2130          |
|    policy_gradient_loss | 0.000385      |
|    reward               | 0.0018485195  |
|    value_loss           | 0.00631       |
-------------------------------------------


-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 9.13e+03      |
|    ep_rew_mean          | 0.852         |
| time/                   |               |
|    fps                  | 227           |
|    iterations           | 216           |
|    time_elapsed         | 8752          |
|    total_timesteps      | 1987200       |
| train/                  |               |
|    approx_kl            | 0.00027510425 |
|    clip_fraction        | 0.00484       |
|    clip_range           | 0.2           |
|    entropy_loss         | -0.109        |
|    explained_variance   | 1.08e-05      |
|    learning_rate        | 0.00025       |
|    loss                 | 0.0014        |
|    n_updates            | 2150          |
|    policy_gradient_loss | 4.4e-05       |
|    reward               | 0.016714098   |
|    value_loss           | 0.00644       |
-------------------------------------------


-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 9.13e+03      |
|    ep_rew_mean          | 0.85          |
| time/                   |               |
|    fps                  | 227           |
|    iterations           | 217           |
|    time_elapsed         | 8791          |
|    total_timesteps      | 1996400       |
| train/                  |               |
|    approx_kl            | 0.00036577502 |
|    clip_fraction        | 0.00396       |
|    clip_range           | 0.2           |
|    entropy_loss         | -0.109        |
|    explained_variance   | 3.16e-06      |
|    learning_rate        | 0.00025       |
|    loss                 | 0.000144      |
|    n_updates            | 2160          |
|    policy_gradient_loss | -0.000125     |
|    reward               | 0.014841702   |
|    value_loss           | 0.00538       |
-------------------------------------------


-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 9.13e+03      |
|    ep_rew_mean          | 0.846         |
| time/                   |               |
|    fps                  | 227           |
|    iterations           | 218           |
|    time_elapsed         | 8831          |
|    total_timesteps      | 2005600       |
| train/                  |               |
|    approx_kl            | 0.00040839636 |
|    clip_fraction        | 0.00477       |
|    clip_range           | 0.2           |
|    entropy_loss         | -0.1          |
|    explained_variance   | 1.29e-05      |
|    learning_rate        | 0.00025       |
|    loss                 | 0.0016        |
|    n_updates            | 2170          |
|    policy_gradient_loss | -0.000271     |
|    reward               | 0.0037693433  |
|    value_loss           | 0.00502       |
-------------------------------------------


Total trained timestep: 2005600


In [ ]:
agent.predict(selected_model,full_env,render=True)

In [ ]:
agent.plot_multiple()